In [21]:
from collections import namedtuple
import csv

bookmarks = []
Bookmark = namedtuple('Bookmark', ['title', 'tags'])

with open('marks.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    # transforms lists as strings to real lists
    for row in reader:
        bookmarks.append(Bookmark(row[0], eval(row[1])))

# prints bookmarks that have compsci as a tag
for bookmark in filter(lambda item: "compsci" in item.tags, bookmarks):
    print(bookmark)

Bookmark(title='“Right click and save as” needs to go away', tags=['compsci'])
Bookmark(title='Fun with Java2D - Strokes', tags=['compsci'])
Bookmark(title='Books for Computer Science Graduate Students', tags=['compsci'])
Bookmark(title='Chrome Extensions Intro', tags=['compsci'])
Bookmark(title="Google's Python Class", tags=['compsci', 'python'])
Bookmark(title='Java Game Framework', tags=['compsci'])
Bookmark(title='musicalgorithms.ewu.edu', tags=['compsci'])
Bookmark(title='Napier Bones', tags=['compsci'])
Bookmark(title='Unity + XMonad', tags=['compsci'])
Bookmark(title='xmonad - Wikipedia, the free encyclopedia', tags=['compsci'])
Bookmark(title='Java2D examples', tags=['compsci', 'java', 'programming'])
Bookmark(title='OpenGL Reference Card', tags=['compsci', 'graphics'])
Bookmark(title='The Art of Rendering', tags=['compsci', 'graphics'])
Bookmark(title='netcat | computer music', tags=['audiodev', 'awesome', 'compsci', 'music'])
Bookmark(title='music21: a Toolkit for Computer-Ai